In [1]:
import wave
import numpy as np
import math
import random

def getData(file):
    if file.getnchannels() == 2:
        my_bytes = file.readframes(1)
        left = int.from_bytes(my_bytes[:file.getsampwidth()], byteorder='little', signed=True)
        right = int.from_bytes(my_bytes[file.getsampwidth():], byteorder='little', signed=True)
        return (left/2) + (right/2)
    else:
        return int.from_bytes(file.readframes(1), byteorder='little', signed=True)

In [2]:
# Read WAV file
def readWav(bn):
    w = wave.open(bn+".wav",'rb')
    parIn = w.getparams()
    dataIn = []
    for i in range(parIn.nframes):
        dataIn.append(getData(w))
    w.close()
    return {'params':parIn, 'data':dataIn}

In [3]:
# Write WAV audio file
# Export amplitudes to text file
def writeWav(wd,bn):
    w = wave.open(bn+".wav","wb")
    t = open(bn+".txt","w")
    w.setparams(wd['params'])
    for i in range(len(wd['data'])):
        o = int(wd['data'][i])
        bts = o.to_bytes(wd['params'].sampwidth,"little",signed=True)
        w.writeframes(bts+bts)
        t.write(str(o)+"\n")
    w.close()
    t.close()

In [4]:
# Write array to file
def writeArray(data,bn):
    t = open(bn+".txt","w")
    for i in range(len(data)):
        t.write(str(i)+","+str(data[i])+"\n")
    t.close()

In [5]:
# Write Fourier Coefficients to file
def writeCoeffs(coeffs,bn):
    t = open(bn+".txt","w")
    t.write("0,"+str(coeffs[0])+",amplitude,frequency (Hz)\n")
    for i in range(1,len(coeffs)):
        t.write(str(i)+","+str(coeffs[i])+","+str(abs(coeffs[i]))+","+str(0.5*44100*i/coeffs[0])+"\n")
    t.close()

In [6]:
# Detect zeros
def zeros(wavdata):
    z = []
    d = wavdata['data']
    for i in range(len(d)-1):
        if d[i]*d[i+1]<0:
            z.append(i)
    return z

In [7]:
# Trim
def trim(wavdata,trimperiod):
    z = []
    dn = wavdata['data']
    i=1
    while dn[i]*dn[i-1]>0:
        i += 1
    periods = (len(dn)-i)//trimperiod
    return {'params':wavdata['params'], 'data':dn[i:i+periods*trimperiod]}

In [8]:
# Normalize
def normalize(wavdata):
    bs = 1111
    dataIn = wavdata['data']
    decay = []
    n = len(dataIn)//bs
    for i in range(n):
        decay.append(abs(np.array(dataIn[i*bs:(i+1)*bs])).mean())
    X = np.array([bs*k for k in range(n)])
    Y = np.array([math.log(decay[j]) for j in range(n)])
    sumX = X.sum()
    sumY = Y.sum()
    sumX2 = (X**2).sum()
    sumY2 = (Y**2).sum()
    sumXY = (X*Y).sum()
    den = n*sumX2 - sumX**2
    b = (sumY*sumX2 - sumX*sumXY)/den
    m = (n*sumXY-sumX*sumY)/den
    return {'params':wavdata['params'], 'data':[dataIn[j]*math.exp(-m*j) for j in range(len(dataIn))]}

In [9]:
# Set average volume (amplitudes) equal
def meter(fromthisone,tothisone):
    w_in = readWav(fromthisone)
    wout = readWav(tothisone)
    avg_in = abs(np.array(w_in['data'])).mean()
    avgout = abs(np.array(wout['data'])).mean()
    ratio = avg_in/avgout
    if (ratio<0.9) or (ratio>1.1):
        print("Amplifying",tothisone+'.wav',"by a factor of",ratio)
        writeWav({'params':wout['params'],'data':ratio*np.array(wout['data'])} ,tothisone+"-amplified")
    else:
        print("Volume ratio",fromthisone+":"+tothisone,"equals",ratio,"-- nothing to do.")

In [10]:
# Add repeats
def addrepeats(wav,number):
    w_in = readWav(wav)
    writeWav({'params':w_in['params'],'data':w_in['data']*number} ,wav+"-"+str(number)+"-repeats")

In [11]:
def bodesrule(f):
    return (7*(f[0]+f[4])+32*(f[1]+f[3])+12*f[2])/22.5
def simpsons38(f):
    return (f[0]+f[3]++3*(f[1]+f[2]))*.375
def simpsons(f):
    return (f[0]+4*f[1]+f[2])/3
def trapezoidal(f):
    return (f[0]+f[1])*.5
def integrate(g):
    bodes = (len(g)-1)//4
    i = 0
    ans = 0
    for j in range(bodes):
        ans += bodesrule(g[i:i+5])
        i += 4
    leftover = len(g)-4*bodes
    k = 4*bodes
    if leftover==2:
        ans += trapezoidal(g[k:])
    elif leftover==3:
        ans += simpsons(g[k:])
    elif leftover==4:
        ans += simpsons38(g[k:])
    return ans

In [19]:
# Compute Fourier coefficients
def FourierCoefficients(wavdata,harmonics):
    l = len(wavdata['data'])
    Fc = [l-1]
    for i in harmonics:
        omega = math.pi*i/(l-1)
        f = [wavdata['data'][j]*math.sin(omega*j) for j in range(l)]
        Fc.append(integrate(f)*2/(l-1))
        if i%1000==0:
            print(str(i),"harmonics calculated...")
    return Fc

In [13]:
# Synthetic Reproduction
def reproduce(fc,whichOfTop,startdata):
    fcz = [0]+fc[1:]
    best = np.flip(np.argsort(abs(np.array(fcz))))
    l = len(startdata)
    for i in whichOfTop:
        k = best[i]
        omega = math.pi*k/(l-1)
        for j in range(l):
            startdata[j] += fc[best[i]]*math.sin(omega*j)
        if i%1000==0:
            print("Top harmonic "+str(i)+" reproduced...")
    return startdata

In [20]:
soundnames = ['ding-medium-2-cycles','e','h','bird1','boilingwater','ding-large','ding-medium','ding-small',
              'hello','LinearAlgebraRules','nail1']
path = "../sounds/"
filenames = [path+s for s in soundnames]
for base in filenames:
    w = readWav(base)
    z = zeros(w)
    t = {'params':w['params'], 'data':w['data'][z[0]:z[len(z)-1]+1]}
    l = len(t['data'])
    b = (min([l/5,22000]))**(1/9)
    tops = [0]+[int(b**k) for k in range(10)]
    print('************************')
    print(base,b,tops[-1])
    print('************************')
    fc = FourierCoefficients(t,range(1,tops[-1]))
    writeArray(t['data'],base+'-trim')
    writeWav(t,base+'-trim')
    writeCoeffs(fc,base+'-FourierCoefficients')
    synthetic = [0]*len(t['data'])
    """
    for k in range(len(tops)-1):
        synthetic = reproduce(fc,range(tops[k],tops[k+1]),synthetic)
        name = base+'fourier'+str(tops[k+1])
        writeArray(synthetic,name)
        writeWav({'params':t['params'], 'data':synthetic},name)"""

************************
../sounds/ding-medium-2-cycles 1.7041204615269103 121
************************
************************
../sounds/e 1.847531909514848 250
************************
************************
../sounds/h 1.8208291050739267 219
************************
************************
../sounds/bird1 2.8596352928291346 12787
************************
1000 harmonics calculated...
2000 harmonics calculated...
3000 harmonics calculated...
4000 harmonics calculated...
5000 harmonics calculated...
6000 harmonics calculated...
7000 harmonics calculated...
8000 harmonics calculated...
9000 harmonics calculated...
10000 harmonics calculated...
11000 harmonics calculated...
12000 harmonics calculated...
************************
../sounds/boilingwater 2.9442302843838255 16624
************************
1000 harmonics calculated...
2000 harmonics calculated...
3000 harmonics calculated...
4000 harmonics calculated...
5000 harmonics calculated...
6000 harmonics calculated...
7000 harmonic

In [15]:
# Adjust volumes
'''
basename = 'LinearAlgebraRules'
numbers = [1,2,8,25,74,217,637,1870,5486,16095]
for n in numbers:
    meter(basename,basename+'fourier'+str(n))
'''

"\nbasename = 'LinearAlgebraRules'\nnumbers = [1,2,8,25,74,217,637,1870,5486,16095]\nfor n in numbers:\n    meter(basename,basename+'fourier'+str(n))\n"

In [16]:
'''
num = [1,2,6,15,37]
num = [92,230,569,1409,3488]
reps = 3
for n in num:
    bn = 'hellofourier'+str(n)
    addrepeats(bn,reps)
'''

"\nnum = [1,2,6,15,37]\nnum = [92,230,569,1409,3488]\nreps = 3\nfor n in num:\n    bn = 'hellofourier'+str(n)\n    addrepeats(bn,reps)\n"